In [128]:
import json
import csv
import pandas as pd
import itertools
from sklearn.naive_bayes import MultinomialNB 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
import collections
import splitter
pd.set_option('display.max_columns', None)
import scipy as sp
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import GridSearchCV
from sklearn import svm


from sklearn.pipeline import Pipeline, FeatureUnion

In [9]:
df = pd.read_json('cards.json')

In [10]:
df.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id',
       'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang',
       'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image',
       'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line',
       'oracle_text', 'power', 'toughness', 'colors', 'color_identity',
       'keywords', 'legalities', 'games', 'reserved', 'foil', 'nonfoil',
       'oversized', 'promo', 'reprint', 'variation', 'set_id', 'set',
       'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri',
       'rulings_uri', 'prints_search_uri', 'collector_number', 'digital',
       'rarity', 'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       'booster', 'story_spotlight', 'edhrec_rank', 'prices', 'related_uris',
       'all_parts', 'promo_types', 'arena_id', 'preview', 'produced_mana',
       'watermark', 'frame_effects', 'loyalty', 'pri

In [11]:
df.head()

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,released_at,uri,scryfall_uri,layout,highres_image,image_status,image_uris,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,games,reserved,foil,nonfoil,oversized,promo,reprint,variation,set_id,set,set_name,set_type,set_uri,set_search_uri,scryfall_set_uri,rulings_uri,prints_search_uri,collector_number,digital,rarity,flavor_text,card_back_id,artist,artist_ids,illustration_id,border_color,frame,full_art,textless,booster,story_spotlight,edhrec_rank,prices,related_uris,all_parts,promo_types,arena_id,preview,produced_mana,watermark,frame_effects,loyalty,printed_name,card_faces,color_indicator,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,variation_of,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,2006-10-06,https://api.scryfall.com/cards/0000579f-7b35-4...,https://scryfall.com/card/tsp/157/fury-sliver?...,normal,True,highres_scan,{'small': 'https://c1.scryfall.com/file/scryfa...,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,[R],[R],[],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,False,False,False,False,c1d109bc-ffd8-428f-8d7d-3f8d7e648046,tsp,Time Spiral,expansion,https://api.scryfall.com/sets/c1d109bc-ffd8-42...,https://api.scryfall.com/cards/search?order=se...,https://scryfall.com/sets/tsp?utm_source=api,https://api.scryfall.com/cards/0000579f-7b35-4...,https://api.scryfall.com/cards/search?order=re...,157,False,uncommon,"""A rift opened, and our arrows were abruptly s...",0aeebaf5-8c7d-4636-9e82-8c27447861f7,Paolo Parente,[d48dd097-720d-476a-8722-6a02854ae28b],2fcca987-364c-4738-a75b-099d8a26d614,black,2003,False,False,True,False,5205.0,"{'usd': '0.96', 'usd_foil': '6.35', 'eur': '0....",{'gatherer': 'https://gatherer.wizards.com/Pag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,2009-10-02,https://api.scryfall.com/cards/00006596-1166-4...,https://scryfall.com/card/zen/21/kor-outfitter...,normal,True,highres_scan,{'small': 'https://c1.scryfall.com/file/scryfa...,{W}{W},2.0,Creature — Kor Soldier,"When Kor Outfitter enters the battlefield, you...",2,2,[W],[W],[],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,False,False,False,False,eb16a2bd-a218-4e4e-8339-4aa1afc0c8d2,zen,Zendikar,expansion,https://api.scryfall.com/sets/eb16a2bd-a218-4e...,https://api.scryfall.com/cards/search?order=se...,https://scryfall.com/sets/zen?utm_source=api,https://api.scryfall.com/cards/00006596-1166-4...,https://api.scryfall.com/cards/search?order=re...,21,False,common,"""We take only what we need to survive. Believe...",0aeebaf5-8c7d-4636-9e82-8c27447861f7,Kieran Yanner,[aa7e89ed-d294-4633-9057-ce04dacfcfa4],de0310d1-e97f-46e0-bc16-c980c2adedee,black,2003,False,False,True,False,10942.0,"{'usd': '0.22', 'usd_foil': '2.19', 'eur': '0....",{'gatherer': 'https://gatherer.wizards.com/Pag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,2015-05-22,https://api.scryfall.com/cards/0000a54c-a511-4...,https://scryfall.com/card/tmm2/5/spirit?utm_so...,token,True,highres_scan,{'small': 'https://c1.scryfall.com/file/scryfa...,,0.0,Token Creature — Spirit,Flying,1,1,[W],[W],[Flying],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,False,True,False,False,True,False,f7aa47c6-c1e2-4de5-9a68-4406d84bd6bb,tmm2,Modern Masters 2015 Tokens,token,https://api.scryfall.com/sets/f7aa47c6-c1e2-4d...,https://api.scryfall.com/cards/search?order=se...,https://scryfall.com/sets/tmm2?utm_source=api,ht

Next, I want to create a new feature that include the card type and subtype (i.e. Creature, Kor Soldier). This can be obtained under type_line. Card type comes before the dash, while subtype comes after, so I will split on the - . 

In [56]:
df['card_type'] = df['type_line'].str.split("—", n = 1, expand = True)[0]
df['subtype'] = df['type_line'].str.split("—", n = 1, expand = True)[1]

In [57]:
card_data = df.to_csv('cards.csv',header=False)

In [58]:
name_strings = df["name"].str.split(" ", n = 10, expand = False)

In [59]:
long_names = []
for i in range(len(name_strings)):
    long_names += name_strings[i]

In [60]:
print(len(set(long_names)))
print(len(long_names))

15661
133285


In [61]:
for i in range(len(name_strings)):
    try:
        name_strings[i].remove('//')
    except ValueError:
        pass
    try:
        name_strings[i].remove('of')
    except ValueError:
        pass
    try:
        name_strings[i].remove('the')
    except ValueError:
        pass
    try:
        name_strings[i].remove("from")
    except ValueError:
        pass
    try:
        name_strings[i].remove("a")
    except ValueError:
        pass
    try:
        name_strings[i].remove("in")
    except ValueError:
        pass
    try:
        name_strings[i].remove("to")
    except ValueError:
        pass

In [62]:
df['name_array'] = name_strings

Now, we want to convert all color identities (G, U, B, W, R, C) as a binary string (i.e. (1 0 0 0 0) for Green, but nothing else, and (1 0 0 0 1) for Gruul, and (0 0 0 0 0 ) for colorless). 

In [63]:
s = df['color_identity']

mlb = MultiLabelBinarizer()

encoded_color_id = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)

In [66]:
new_df = df[['name_array', 'color_identity', 'cmc', 'keywords', 'card_type', 'subtype']].copy()
new_df['B'] = encoded_color_id['B']
new_df['U'] = encoded_color_id['U']
new_df['W'] = encoded_color_id['W']
new_df['G'] = encoded_color_id['G']
new_df['R'] = encoded_color_id['R']
#new_df['C'] = encoded_color_id['C']

In [67]:
new_df

,name_array,color_identity,cmc,keywords,card_type,subtype,B,U,W,G,R
0,"[Fury, Sliver]",[R],6.0,[],Creature,Sliver,0,0,0,0,1
1,"[Kor, Outfitter]",[W],2.0,[],Creature,Kor Soldier,0,0,1,0,0
2,[Spirit],[W],0.0,[Flying],Token Creature,Spirit,0,0,1,0,0
3,"[Siren, Lookout]",[U],3.0,"[Flying, Explore]",Creature,Siren Pirate,0,1,0,0,0
4,[Web],[G],1.0,[Enchant],Enchantment,Aura,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
61588,"[Angel's, Tomb]",[],3.0,[],Artifact,None,0,0,0,0,0
61589,[Boomerang],[U],2.0,[],Instant,None,0,1,0,0,0
61590,"[Celestine, Reef]",[],0.0,[],Plane,Luvion,0,0,0,0,0
61591,"[Horned, Troll]",[G],3.0,[],Creature,Troll,0,0,0,1,0


In [68]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
X, y = make_multilabel_classification(n_classes=3, random_state=0)

In [69]:
Y = new_df[['B', 'W', 'U', 'R', 'G']]
X = new_df[['name_array']]
new_df['names'] = 0

I tried splitting the words using the 'splitter' package but unfortunately that didn't lead to anything useful. While some cards like "Worldfire" would be split into "World fire" (which is useful), other names like "Beledros" would be split into "Be le dr os" (this is actively bad). 

In [95]:
j = 0
for i in new_df['name_array']:
    i = ' '.join(i) #Uncomment if not using splitter
    new_df['names'][j] = i
    j += 1
new_df.head()

c:\users\emily\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,name_array,color_identity,cmc,keywords,card_type,subtype,B,U,W,G,R,names
0,"[Fury, Sliver]",[R],6.0,[],Creature,Sliver,0,0,0,0,1,Fury Sliver
1,"[Kor, Outfitter]",[W],2.0,[],Creature,Kor Soldier,0,0,1,0,0,Kor Outfitter
2,[Spirit],[W],0.0,[Flying],Token Creature,Spirit,0,0,1,0,0,Spirit
3,"[Siren, Lookout]",[U],3.0,"[Flying, Explore]",Creature,Siren Pirate,0,1,0,0,0,Siren Lookout
4,[Web],[G],1.0,[Enchant],Enchantment,Aura,0,0,0,1,0,Web


In [108]:
new_df.subtype = new_df.subtype.astype(str)

In [133]:
vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(new_df['names'])
#X = vectorizer.fit_transform(new_df['names'] + new_df['cmc'] +  new_df['keywords'])

In [134]:
vectorizer.fit_transform(new_df.names)


<61593x14417 sparse matrix of type '<class 'numpy.int64'>'
	with 120676 stored elements in Compressed Sparse Row format>

In [135]:
mapper = DataFrameMapper([
    ('card_type', CountVectorizer()),
    ('names',CountVectorizer()), ('subtype', CountVectorizer())])
X = mapper.fit_transform(new_df)

In [136]:
pipeline = Pipeline([
    ('vectorizer',
        mapper),
    ('classifier', MultinomialNB())
])

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) 
# 64% train, 16% validation and 20% test 

When I include colorless as its own distinct color, as opposed to purely viewing it as a lack of color, the accuracy goes down by ~2.5 percent to 0.6756. One way to mitigate this issue might be to count a prediction of [0 0 0 0 0 0] and [0 0 0 0 0 1] both as colorless. 

Making this change boosted it to 0.6971881290992922, but ultimately, this means it's still better to view colorless as the absence of other colors. 

In [138]:
multinb = MultinomialNB()
naive_bayes = MultiOutputClassifier(multinb)
y_pred = naive_bayes.fit(X_train, y_train).predict(X_val)
accuracy_score(y_val, y_pred)

0.6997142609749762

In [ ]:
svc_clf = svm.SVC()
svc_model = MultiOutputClassifier(svc_clf)
y_pred = svc_model.fit(X_train, y_train).predict(X_val)
accuracy_score(y_val, y_pred)

In [ ]:
for i in range(15399):
    if(np.any(y_test.iloc[i] != y_pred_df.iloc[i])):
        print(new_df['names'][y_test.iloc[i].name])
        print("Predicted \n",  y_pred_df.iloc[i])
        print("Actual \n", y_test.iloc[i], "\n")

I noticed that it predicted "Elves" as green, but in actuality it was colorless. This didn't make sense to me (I would also expect it to be green) so I checked Scryfall, and it turns out that it was a Jumpstart front card. This informs my feature engineering because it drew attention to the fact that there are cards in the dataset that are actually just cosmetic, and should be removed. 

The cards it was tripping up on were often compound words or hyphenated, and I expect the results to be improved if I can something separate words by substrings. 

In [ ]:
df.iloc[45788]['scryfall_uri']

In [ ]:
splitter.split('worldfire')

In [ ]:
' '.join(['world', 'fire'])

In [ ]:
df[['name', 'colors']]